### PyEmma Featurizer Support

In [1]:
import openpathsampling as ps
import openpathsampling.storage as st

In [2]:
#! lazy
import pyemma.coordinates as coor

In [3]:
storage = st.Storage('trajectory.nc', mode='a')

Import a PyEmma Coordinates Module

Using of pyemma featurizers or general other complex code requires a little trick to be storable. Since storing of code only works if we are not dependend on the context (scope) we need to wrap the construction of our featurizer in a function, that gets all it needs from the global scope as a parameter

In [4]:
def pyemma_generator(f):
    f.add_inverse_distances(f.pairs(f.select_Backbone()))

In [5]:
cv = ps.collectivevariable.CV_PyEMMA_Featurizer('pyemma', pyemma_generator, topology=storage.template.topology)

Now use this featurizer generating function to build a collective variable out of it. All we need for that is a name as usual, the generating function, the list of parameters - here only the topology and at best a test snapshot, a template.

In [6]:
cv(storage.trajectories[0]);

Let's save it to the storage

In [7]:
cv.save(storage)

and apply the featurizer to a trajectory

In [8]:
cv(storage.trajectories[2]);

Sync to make sure the cache is written to the netCDF file.

In [9]:
storage.sync_all()

In [10]:
cv(storage.snapshots.all());

In [11]:
storage.cvs.sync()

In [12]:
storage.sync_all()

In [13]:
py_cv = storage.cvs['pyemma']

In [14]:
nc_var = storage.variables['snapshots_cv_%d_values' % storage.idx(py_cv)]

In [15]:
assert(nc_var.shape[1] == 15)
print nc_var.shape[1]

15


In [16]:
assert(nc_var.var_type == 'numpy.float32')
print nc_var.var_type

numpy.float32


In [17]:
#! ignore
print storage.variables['cvs_json'][:]

[ u'{"_cls": "CV_Function", "_dict": {"store_cache": true, "name": "psi2", "f": {"_marshal": "YwMAAAADAAAAAwAAAEMAAABzHAAAAHwBAHwAAIMBAGQBABN8AgB8AACDAQBkAQATF1MoAgAAAE5pAgAAACgAAAAAKAMAAAB0BAAAAGl0ZW10AwAAAHBzaXQDAAAAcGhpKAAAAAAoAAAAAHMeAAAAPGlweXRob24taW5wdXQtOS00NGFmZmMyZGY1MzU+dAcAAABwcDJfZm5jAQAAAHMCAAAAAAE=", "_module_vars": [], "_global_vars": []}, "scalarize_numpy_singletons": false, "wrap_numpy_array": false, "kwargs": {"phi": {"_obj": "cvs", "_idx": 1}, "psi": {"_obj": "cvs", "_idx": 2}}, "requires_lists": false, "time_reversible": false}}'
 u'{"_cls": "CV_MDTraj_Function", "_dict": {"store_cache": true, "name": "phi", "f": {"_module": "mdtraj.geometry.dihedral", "_name": "compute_dihedrals"}, "scalarize_numpy_singletons": true, "wrap_numpy_array": true, "kwargs": {"indices": [[4, 6, 8, 14]]}, "requires_lists": true, "time_reversible": true}}'
 u'{"_cls": "CV_MDTraj_Function", "_dict": {"store_cache": true, "name": "psi", "f": {"_module": "mdtraj.geometry.dihedral", "_name": 

In [18]:
py_emma_feat = storage.vars['cvs_json'][6]

In [19]:
erg = py_emma_feat(storage.snapshots);

In [20]:
#! lazy
print erg[0::10,1]

[ 4.05474138  4.04429817  4.09497976  4.05629969  4.02169132  4.08048439
  3.99784589  4.06264544  3.99345803  4.06143045  4.06058693  4.01164579
  4.07558966  4.00137377  4.09326935  4.01662636  4.01421356  4.04161978
  4.02600718  4.00666857  4.06887293  4.05741262  4.0694766   4.02478552
  4.05429459  3.99438739  3.99598861  4.0506835   3.95376158]
